In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

In [2]:
BASE_DIR = Path('../../../')

CSV_DIR = BASE_DIR / 'data' / 'iemocap_4way_data'
TRAIN_CSV_PATH = CSV_DIR / 'train_4way_unified.csv'
VAL_CSV_PATH = CSV_DIR / 'val_4way_unified.csv'
TEST_CSV_PATH = CSV_DIR / 'test_4way_unified.csv'

EMBEDDING_DIR = BASE_DIR / 'data' / 'embeddings' / '4way' / 'opensmile' / 'eGeMAPSv02'
TRAIN_EMBEDDING_PATH = EMBEDDING_DIR / 'train_unified.npy'
VAL_EMBEDDING_PATH = EMBEDDING_DIR / 'val_unified.npy'
TEST_EMBEDDING_PATH = EMBEDDING_DIR / 'test_unified.npy'

### Filter openSMILE Embeddings

In [3]:
def filter_embeddings(csv_path, embedding_path, output_path):
    # Load CSV and embeddings
    df = pd.read_csv(csv_path)
    embeddings = np.load(embedding_path, allow_pickle=True)

    # Edit embeddings so that it matches the CSV files. df['id'] corresponds to the second column of the embeddings array
    # Create a filtered embeddings array where the order of the rows matches the order of df['id'] in the CSV file. In the filtered array, only keep the first column
    filtered_embeddings = []
    id_to_embedding = {row[1]: row[0] for row in embeddings}
    valid_indices = []
    for i, filename in enumerate(df['id']):
        try:
            filtered_embeddings.append(id_to_embedding[filename])
            valid_indices.append(i)
        except KeyError:
            pass
    filtered_embeddings = np.array(filtered_embeddings)

    # Save filtered embeddings
    np.save(output_path, filtered_embeddings)

    return valid_indices

#### Filter Training, Validation, and Testing Data Embeddings

In [4]:
valid_train = filter_embeddings(TRAIN_CSV_PATH, TRAIN_EMBEDDING_PATH, EMBEDDING_DIR / 'train_unified_filtered.npy')
valid_val = filter_embeddings(VAL_CSV_PATH, VAL_EMBEDDING_PATH, EMBEDDING_DIR / 'val_unified_filtered.npy')
valid_test = filter_embeddings(TEST_CSV_PATH, TEST_EMBEDDING_PATH, EMBEDDING_DIR / 'test_unified_filtered.npy')

### Filter SRoBERTa Embeddings

In [5]:
def filter_sroberta_embeddings(npz_path, valid_indices, output_path):
    # Load the embeddings
    data = np.load(npz_path)
    embeddings = data['embeddings']

    # Ensure indices are a NumPy array
    valid_indices = np.asarray(valid_indices, dtype=int)

    # Filter the embeddings
    filtered_embeddings = embeddings[valid_indices]

    # Save to output path
    np.savez(output_path, embeddings=filtered_embeddings)

    print(f"Filtered embeddings saved to {output_path} with shape {filtered_embeddings.shape}")


In [6]:
SROBERTA_DIR = BASE_DIR / 'data' / 'embeddings' / '4way/sroberta/sroberta-hier-avglast4-mean'
SROBERTA_TRAIN_PATH = SROBERTA_DIR / 'train.npz'
SROBERTA_VAL_PATH = SROBERTA_DIR / 'val.npz'
SROBERTA_TEST_PATH = SROBERTA_DIR / 'test.npz'

In [7]:
filter_sroberta_embeddings(SROBERTA_TRAIN_PATH, valid_train, SROBERTA_DIR / 'train_filtered.npz')

Filtered embeddings saved to ../../../data/embeddings/4way/sroberta/sroberta-hier-avglast4-mean/train_filtered.npz with shape (6050, 12, 768)


In [8]:
filter_sroberta_embeddings(SROBERTA_VAL_PATH, valid_val, SROBERTA_DIR / 'val_filtered.npz')

Filtered embeddings saved to ../../../data/embeddings/4way/sroberta/sroberta-hier-avglast4-mean/val_filtered.npz with shape (1819, 10, 768)


In [9]:
filter_sroberta_embeddings(SROBERTA_TEST_PATH, valid_test, SROBERTA_DIR / 'test_filtered.npz')

Filtered embeddings saved to ../../../data/embeddings/4way/sroberta/sroberta-hier-avglast4-mean/test_filtered.npz with shape (2170, 12, 768)


### Filter CSV Files

In [10]:
def filter_csv_with_indices(csv_path, valid_indices, output_path):
    """Filter a CSV by row indices and write to output_path."""
    df = pd.read_csv(csv_path)
    valid_indices = np.asarray(valid_indices, dtype=int)
    # Keep rows in the original order specified by valid_indices
    filtered_df = df.iloc[valid_indices].reset_index(drop=True)
    filtered_df.to_csv(output_path, index=False)
    print(f"Saved filtered CSV to {output_path} with {len(filtered_df)} rows")

# Write filtered CSVs using the indices returned by filter_embeddings above
filter_csv_with_indices(TRAIN_CSV_PATH, valid_train, CSV_DIR / 'train_unified_filtered.csv')
filter_csv_with_indices(VAL_CSV_PATH, valid_val, CSV_DIR / 'val_unified_filtered.csv')
filter_csv_with_indices(TEST_CSV_PATH, valid_test, CSV_DIR / 'test_unified_filtered.csv')

Saved filtered CSV to ../../../data/iemocap_4way_data/train_unified_filtered.csv with 6050 rows
Saved filtered CSV to ../../../data/iemocap_4way_data/val_unified_filtered.csv with 1819 rows
Saved filtered CSV to ../../../data/iemocap_4way_data/test_unified_filtered.csv with 2170 rows
